In [1]:
import numpy as np
import pandas as pd
from pygmid import Lookup as lk

# Technology data - using your paths
n = lk('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/nfet_03v3.mat')
p = lk('/foss/designs/SSCS-Chipathon-2025-DAC-for-the-future/DESIGN/DACFUTURE_EDU/sweep/sweep_script/pfet_03v3.mat')

# Helper function to handle list/scalar returns from lookup
def get_value(lookup_result):
    """Extract scalar value from lookup result"""
    if lookup_result is None or (isinstance(lookup_result, list) and len(lookup_result) == 0):
        return None
    if isinstance(lookup_result, np.ndarray):
        # Handle 0-dimensional arrays (scalars)
        if lookup_result.ndim == 0:
            return float(lookup_result.item())
        # Handle 1-dimensional arrays
        elif lookup_result.size > 0:
            return float(lookup_result[0])
        else:
            return None
    elif isinstance(lookup_result, list) and len(lookup_result) > 0:
        return float(lookup_result[0])
    else:
        return float(lookup_result)

# ==================== SPECIFICATIONS ====================
# Given specifications
fu_cl = 2e6        # Closed-loop bandwidth = 2MHz
CL = 20e-12        # Load capacitance = 20pF
Av_target = 100000 # DC gain = 100,000 (100dB)
Vcm = 1.65         # Common mode voltage = 1.65V
Rout_spec = 500    # Output impedance = 500Ω
VDD = 3.3          # Supply voltage
VSS = 0            # Ground

print("=== TWO-STAGE OTA DESIGN WITH MILLER COMPENSATION ===\n")

# ==================== DESIGN CHOICES ====================
# gm/ID ratios 
gm_id0 = 10    # M0 - tail current
gm_id1 = 12    # M1 - input pair
gm_id2 = 10    # M2 - NMOS mirror
gm_id5 = 12    # M5 - CS stage
gm_id6 = 10    # M6 - PMOS load

# Initial channel lengths
l0 = 0.5   # Tail current
l1 = 0.5   # Input pair
l2 = 1.0   # NMOS mirror
l5 = 0.5   # CS stage
l6 = 1.0   # PMOS load

# Compensation factor
Cc_factor = 0.3  

# ==================== DESIGN CALCULATIONS ====================
print("=== DESIGN PROCESS ===\n")

# Step 1: Calculate compensation capacitor
Cc = Cc_factor * CL
print(f"1. Compensation capacitor: Cc = {Cc_factor}×CL = {Cc*1e12:.1f}pF")

# Step 2: Calculate gm1 from unity gain frequency
GBW = fu_cl  # Unity gain frequency
gm1 = 2 * np.pi * GBW * Cc
print(f"\n2. First stage transconductance:")
print(f"   gm1 = 2π×GBW×Cc = {gm1*1e3:.2f}mS")

# Step 3: Calculate gm2 for stability
k = 10  # Higher ratio for better stability
gm2 = k * gm1
print(f"\n3. Second stage transconductance:")
print(f"   gm2 = {k}×gm1 = {gm2*1e3:.2f}mS")

# Step 4: Calculate currents
id1 = gm1 / gm_id1  # Total current for diff pair
id0 = id1           # Total tail current (equals sum of diff pair)
id2 = id1 / 2       # Current per transistor in diff pair and mirror
id5 = gm2 / gm_id5  # CS stage current
id6 = id5           # Load current

print(f"\n4. Branch currents:")
print(f"   I_tail (M0) = {id0*1e6:.1f}μA")
print(f"   I_M1a = I_M1b = I_M2a = I_M2b = {id2*1e6:.1f}μA (each)")
print(f"   I_M5 = I_M6 = {id5*1e6:.1f}μA")

# Step 5: Size transistors using lookup tables
print(f"\n5. Transistor sizing:")

# M0 - PMOS tail current mirror
jd0_lookup = p.lookup('ID_W', GM_ID=gm_id0, L=l0)
jd0 = get_value(jd0_lookup)
if jd0 is None or jd0 <= 0:
    print(f"   ERROR: Invalid ID_W for M0: {jd0_lookup}")
    jd0 = 1e-5  # Default value
w0 = id0 / jd0
print(f"   M0: W = {w0:.1f}μm, L = {l0}μm, ID/W = {jd0*1e6:.1f}μA/μm")

# M1 - PMOS input pair
jd1_lookup = p.lookup('ID_W', GM_ID=gm_id1, L=l1)
jd1 = get_value(jd1_lookup)
if jd1 is None or jd1 <= 0:
    print(f"   ERROR: Invalid ID_W for M1: {jd1_lookup}")
    jd1 = 1e-5
w1 = id2 / jd1  # Width per transistor
print(f"   M1a,M1b: W = {w1:.1f}μm, L = {l1}μm (each), ID/W = {jd1*1e6:.1f}μA/μm")

# M2 - NMOS current mirror
jd2_lookup = n.lookup('ID_W', GM_ID=gm_id2, L=l2)
jd2 = get_value(jd2_lookup)
if jd2 is None or jd2 <= 0:
    print(f"   ERROR: Invalid ID_W for M2: {jd2_lookup}")
    jd2 = 1e-5
w2 = id2 / jd2  # Width per transistor
print(f"   M2a,M2b: W = {w2:.1f}μm, L = {l2}μm (each), ID/W = {jd2*1e6:.1f}μA/μm")

# M5 - NMOS CS stage
jd5_lookup = n.lookup('ID_W', GM_ID=gm_id5, L=l5)
jd5 = get_value(jd5_lookup)
if jd5 is None or jd5 <= 0:
    print(f"   ERROR: Invalid ID_W for M5: {jd5_lookup}")
    jd5 = 1e-5
w5 = id5 / jd5
print(f"   M5: W = {w5:.1f}μm, L = {l5}μm, ID/W = {jd5*1e6:.1f}μA/μm")

# M6 - PMOS load
jd6_lookup = p.lookup('ID_W', GM_ID=gm_id6, L=l6)
jd6 = get_value(jd6_lookup)
if jd6 is None or jd6 <= 0:
    print(f"   ERROR: Invalid ID_W for M6: {jd6_lookup}")
    jd6 = 1e-5
w6 = id6 / jd6
print(f"   M6: W = {w6:.1f}μm, L = {l6}μm, ID/W = {jd6*1e6:.1f}μA/μm")

# Step 6: Calculate gains
print(f"\n6. Gain calculation:")

# First stage gain
gm_gds1_lookup = p.lookup('GM_GDS', GM_ID=gm_id1, L=l1)
gm_gds1 = get_value(gm_gds1_lookup)
if gm_gds1 is None or gm_gds1 <= 0:
    print(f"   ERROR: Invalid GM_GDS for M1: {gm_gds1_lookup}")
    gm_gds1 = 100  # Default

gm_gds2_lookup = n.lookup('GM_GDS', GM_ID=gm_id2, L=l2)
gm_gds2 = get_value(gm_gds2_lookup)
if gm_gds2 is None or gm_gds2 <= 0:
    print(f"   ERROR: Invalid GM_GDS for M2: {gm_gds2_lookup}")
    gm_gds2 = 100  # Default

# Calculate stage 1 gain
gm1_total = gm_id1 * id2 * 2  # Total gm of diff pair
gds1 = (gm_id1 * id2) / gm_gds1  # Output conductance of one M1
gds2 = (gm_id2 * id2) / gm_gds2  # Output conductance of one M2
ro1 = 1 / (gds1 + gds2)
Av1 = gm1_total * ro1
print(f"   First stage: gm1={gm1_total*1e3:.2f}mS, ro1={ro1/1e3:.1f}kΩ")
print(f"   Av1 = {Av1:.0f} ({20*np.log10(Av1):.1f}dB)")

# Second stage gain
gm_gds5_lookup = n.lookup('GM_GDS', GM_ID=gm_id5, L=l5)
gm_gds5 = get_value(gm_gds5_lookup)
if gm_gds5 is None or gm_gds5 <= 0:
    print(f"   ERROR: Invalid GM_GDS for M5: {gm_gds5_lookup}")
    gm_gds5 = 100

gm_gds6_lookup = p.lookup('GM_GDS', GM_ID=gm_id6, L=l6)
gm_gds6 = get_value(gm_gds6_lookup)
if gm_gds6 is None or gm_gds6 <= 0:
    print(f"   ERROR: Invalid GM_GDS for M6: {gm_gds6_lookup}")
    gm_gds6 = 100

gm2_actual = gm_id5 * id5
gds5 = gm2_actual / gm_gds5
gds6 = (gm_id6 * id6) / gm_gds6
ro2 = 1 / (gds5 + gds6)
Av2 = gm2_actual * ro2
print(f"   Second stage: gm2={gm2_actual*1e3:.2f}mS, ro2={ro2/1e3:.1f}kΩ")
print(f"   Av2 = {Av2:.0f} ({20*np.log10(Av2):.1f}dB)")

# Total gain
Av_total = Av1 * Av2
print(f"   Total: Av = {Av_total:.0f} ({20*np.log10(Av_total):.1f}dB)")

# Auto-increase L if gain is insufficient
if Av_total < Av_target:
    print(f"\n7. Auto-sizing L to meet gain spec...")
    iteration = 0
    
    while Av_total < Av_target and iteration < 10:
        iteration += 1
        # Increase L for high-gain devices
        l2 = l2 * 1.5
        l6 = l6 * 1.5
        
        print(f"   Iteration {iteration}: L2={l2:.1f}μm, L6={l6:.1f}μm")
        
        # Recalculate sizes
        jd2 = get_value(n.lookup('ID_W', GM_ID=gm_id2, L=l2))
        if jd2 is None or jd2 <= 0:
            jd2 = 1e-5
        w2 = id2 / jd2
        
        jd6 = get_value(p.lookup('ID_W', GM_ID=gm_id6, L=l6))
        if jd6 is None or jd6 <= 0:
            jd6 = 1e-5
        w6 = id6 / jd6
        
        # Recalculate gains
        gm_gds2 = get_value(n.lookup('GM_GDS', GM_ID=gm_id2, L=l2))
        if gm_gds2 is None or gm_gds2 <= 0:
            gm_gds2 = 100
        
        gm_gds6 = get_value(p.lookup('GM_GDS', GM_ID=gm_id6, L=l6))
        if gm_gds6 is None or gm_gds6 <= 0:
            gm_gds6 = 100
        
        gds2 = (gm_id2 * id2) / gm_gds2
        ro1 = 1 / (gds1 + gds2)
        Av1 = gm1_total * ro1
        
        gds6 = (gm_id6 * id6) / gm_gds6
        ro2 = 1 / (gds5 + gds6)
        Av2 = gm2_actual * ro2
        
        Av_total = Av1 * Av2
        print(f"   New gain: {Av_total:.0f} ({20*np.log10(Av_total):.1f}dB)")
        
        if Av_total >= Av_target:
            print(f"   Target gain achieved!")
            break

# Output impedance
Rout = ro2
print(f"\n8. Output impedance: Rout = {Rout/1e3:.1f}kΩ")

# ==================== STABILITY ANALYSIS ====================
print("\n=== STABILITY ANALYSIS ===")

# Dominant pole (Miller effect)
fp1 = 1 / (2 * np.pi * Av2 * Cc * ro1)
print(f"Dominant pole: fp1 = {fp1:.1f}Hz")

# Second pole
fp2_actual = gm2_actual / (2 * np.pi * CL)
print(f"Second pole: fp2 = {fp2_actual/1e6:.1f}MHz")
print(f"fp2/GBW = {fp2_actual/GBW:.1f} (should be > 2.2)")

# RHP zero
fz = gm2_actual / (2 * np.pi * Cc)
print(f"RHP zero: fz = {fz/1e6:.1f}MHz")

# Phase margin
phase_p2 = -np.arctan(GBW/fp2_actual) * 180/np.pi
phase_z = -np.arctan(GBW/fz) * 180/np.pi
PM = 90 + phase_p2 + phase_z
print(f"Phase margin: PM = {PM:.1f}°")

# ==================== DEVICE FINGERING ====================
wfing = 5  # Finger width
nf0 = int(1 + np.floor(w0/wfing))
nf1 = int(1 + np.floor(w1/wfing))
nf2 = int(1 + np.floor(w2/wfing))
nf5 = int(1 + np.floor(w5/wfing))
nf6 = int(1 + np.floor(w6/wfing))

# ==================== SUMMARY TABLES ====================
print("\n=== FINAL TRANSISTOR SIZES ===")
df_size = pd.DataFrame({
    'Device': ['M0', 'M1a,M1b', 'M2a,M2b', 'M5', 'M6'],
    'Type': ['PMOS', 'PMOS', 'NMOS', 'NMOS', 'PMOS'],
    'ID (μA)': [f'{id0*1e6:.1f}', f'{id2*1e6:.1f}', f'{id2*1e6:.1f}', 
                f'{id5*1e6:.1f}', f'{id6*1e6:.1f}'],
    'W (μm)': [f'{w0:.1f}', f'{w1:.1f}', f'{w2:.1f}', f'{w5:.1f}', f'{w6:.1f}'],
    'L (μm)': [f'{l0:.1f}', f'{l1:.1f}', f'{l2:.1f}', f'{l5:.1f}', f'{l6:.1f}'],
    'nf': [nf0, nf1, nf2, nf5, nf6],
    'W/L': [f'{w0/l0:.1f}', f'{w1/l1:.1f}', f'{w2/l2:.1f}', 
            f'{w5/l5:.1f}', f'{w6/l6:.1f}']
})
print(df_size.to_string(index=False))

print("\n=== PERFORMANCE SUMMARY ===")
print(f"Unity Gain Frequency: {GBW/1e6:.2f} MHz")
print(f"DC Gain: {20*np.log10(Av_total):.1f} dB")
print(f"Phase Margin: {PM:.1f}°")
print(f"Output Impedance: {Rout/1e3:.1f} kΩ")
print(f"Power Consumption: {(id0+id5)*3.3*1e6:.1f} μW")
print(f"Compensation Cap: {Cc*1e12:.1f} pF")

print("\n=== M6 BIAS CONNECTION ===")
print("M6 can be connected to the same current mirror as M0:")
print(f"- M0 provides {id0*1e6:.1f}μA (tail current)")
print(f"- M6 needs {id6*1e6:.1f}μA (second stage current)")
print(f"- Mirror ratio M6/M0 = {id6/id0:.2f}")
print(f"- W6_adjusted = W0 × {id6/id0:.2f} = {w0*id6/id0:.1f}μm")

# ==================== WRITE SPICE NETLIST ====================
with open('sizing_ota-2stage_gf.spice', 'w') as file:
    file.write("* Two-Stage OTA with Miller Compensation\n")
    file.write("* GF180MCU Process\n\n")
    
    file.write(f".param ibn = {id0:.2e}\n")
    file.write(f".param W0 = {w0*1e-6:.2e}\n")
    file.write(f".param W1 = {w1*1e-6:.2e}\n")
    file.write(f".param W2 = {w2*1e-6:.2e}\n")
    file.write(f".param W5 = {w5*1e-6:.2e}\n")
    file.write(f".param W6 = {w6*1e-6:.2e}\n")
    file.write(f".param L0 = {l0*1e-6:.2e}\n")
    file.write(f".param L1 = {l1*1e-6:.2e}\n")
    file.write(f".param L2 = {l2*1e-6:.2e}\n")
    file.write(f".param L5 = {l5*1e-6:.2e}\n")
    file.write(f".param L6 = {l6*1e-6:.2e}\n")
    file.write(f".param nf0 = {nf0}\n")
    file.write(f".param nf1 = {nf1}\n")
    file.write(f".param nf2 = {nf2}\n")
    file.write(f".param nf5 = {nf5}\n")
    file.write(f".param nf6 = {nf6}\n")
    #file.write(f".param Cc = {Cc:.2e}\n")

print("\nSPICE netlist written to 'sizing_ota-2stage_gf.spice'")

=== TWO-STAGE OTA DESIGN WITH MILLER COMPENSATION ===

=== DESIGN PROCESS ===

1. Compensation capacitor: Cc = 0.3×CL = 6.0pF

2. First stage transconductance:
   gm1 = 2π×GBW×Cc = 0.08mS

3. Second stage transconductance:
   gm2 = 10×gm1 = 0.75mS

4. Branch currents:
   I_tail (M0) = 6.3μA
   I_M1a = I_M1b = I_M2a = I_M2b = 3.1μA (each)
   I_M5 = I_M6 = 62.8μA

5. Transistor sizing:
   M0: W = 5.0μm, L = 0.5μm, ID/W = 1.3μA/μm
   M1a,M1b: W = 4.0μm, L = 0.5μm (each), ID/W = 0.8μA/μm
   M2a,M2b: W = 1.4μm, L = 1.0μm (each), ID/W = 2.2μA/μm
   M5: W = 23.1μm, L = 0.5μm, ID/W = 2.7μA/μm
   M6: W = 120.4μm, L = 1.0μm, ID/W = 0.5μA/μm

6. Gain calculation:
   First stage: gm1=0.08mS, ro1=4685.1kΩ
   Av1 = 353 (51.0dB)
   Second stage: gm2=0.75mS, ro2=190.2kΩ
   Av2 = 143 (43.1dB)
   Total: Av = 50650 (94.1dB)

7. Auto-sizing L to meet gain spec...
   Iteration 1: L2=1.5μm, L6=1.5μm
   New gain: 58487 (95.3dB)
   Iteration 2: L2=2.2μm, L6=2.2μm
   New gain: 66000 (96.4dB)
   Iteration 3: L2